In [34]:
import sys 
import os
import json_tricks as json
dotenv_path = os.path.abspath("../../backend/.env")
if os.path.exists(dotenv_path):
    from dotenv import load_dotenv
    load_dotenv(dotenv_path)
    
sys.path.append("../..")
sys.path.append("../../..")
from backend.LLM.AnythingLLM_client import AnythingLLMClient
from backend.Generater import Generater
from backend.LLM.OllamaLLM import OllamaAI
from backend.database.schemas import Email


anyllm_client = AnythingLLMClient("http://localhost:3001/api", "3WMNAPZ-GYH4RBE-M67SR00-7Y7KYEF")
ollama_client = OllamaAI('http://localhost:11434', 'llama3:instruct')
generator = Generater(ollama_client, anyllm_client)
path_to_dataset = os.path.abspath("../../dataset")


Performing health check on http://localhost:11434
Ollama is running
Health check passed


# Play with temperature

In [ ]:

query_refuse = "Cannot answer"
workspace_slug = anyllm_client.get_workspace_slug("temperature_play")

for enquiry in os.listdir(path_to_dataset)[:1]:
    if not enquiry.endswith(".json") or not "fake" in enquiry:
        continue
        
    with open(os.path.join(path_to_dataset, enquiry), "r") as f:
        data = json.load(f)
        
    questions = data["questions"]
    results = []
    for i in range(len(questions)):
        question = data["questions"][i]["question"]
        temperature = 1
        for reducer in range(0, 10, 1):
            temperature = round(1 - round(reducer/10, 1), 1)
            print(f"Temperature: {temperature}")
            anyllm_client.update_workspace(workspace_slug, { "openAiTemp": temperature })
            new_thread = anyllm_client.new_thread(workspace_slug, f"temp_{temperature}")
            new_thread_slug = new_thread["slug"]
            res = anyllm_client.chat_with_thread(workspace_slug, new_thread_slug, question)
            results.append({
                "temperature": temperature,
                "response": res["textResponse"]
            })
            anyllm_client.delete_thread(workspace_slug, new_thread_slug)
            
path_results = "results.json"
with open(path_results, "w") as f:
    json.dump(results, f)

# Similarity Threshold

In [ ]:

query_refuse = "Cannot answer"
workspace_slug = anyllm_client.get_workspace_slug("temperature_play")

for enquiry in os.listdir(path_to_dataset)[:1]:
    if not enquiry.endswith(".json") or not "fake" in enquiry:
        continue
        
    with open(os.path.join(path_to_dataset, enquiry), "r") as f:
        data = json.load(f)
        
    questions = data["questions"]
    results = []
    for i in range(len(questions)):
        question = data["questions"][i]["question"]

        for reducer in range(4, -1, -1):
            similarityThreshold = 1 - (reducer/4) 
            print(f"similarityThreshold: {similarityThreshold}")
            anyllm_client.update_workspace(workspace_slug, { "similarityThreshold": similarityThreshold })
            new_thread = anyllm_client.new_thread(workspace_slug, f"sim_{similarityThreshold}")
            new_thread_slug = new_thread["slug"]
            res = anyllm_client.chat_with_thread(workspace_slug, new_thread_slug, question)
            results.append({
                "similarityThreshold": similarityThreshold,
                "response": res["textResponse"],
                "sources": res["sources"]
            })
            anyllm_client.delete_thread(workspace_slug, new_thread_slug)
            
path_results = "results_sim.json"
with open(path_results, "w") as f:
    json.dump(results, f)

# Making Rules

In [47]:
from datetime import datetime

rules = [
    "Call by their last name",
    "Change the tone to be professional",
    "Greetings depend on the time of the day",
]

paramters = {
    "today": datetime.now().strftime("%Y-%m-%d"),
    "rules": rules
}

In [48]:

query_refuse = "Cannot answer"
workspace_slug = anyllm_client.get_workspace_slug("temperature_play")
anyllm_client.update_workspace(workspace_slug, { "similarityThreshold": 0.5 })

for enquiry in os.listdir(path_to_dataset)[:1]:
    if not enquiry.endswith(".json") or not "fake" in enquiry:
        continue
        
    with open(os.path.join(path_to_dataset, enquiry), "r") as f:
        data = json.load(f)
        
    results = []
    email = data["email"]
    
    prompt= f"""
        You are David Frischer, the program administrator for the Software Engineering program.
        You have received an email from a student:
        
        {email}
        
        Reply to the student's email with the following rules:
        {json.dumps(paramters)}
        
        """
        
    

    new_thread = anyllm_client.new_thread(workspace_slug, f"rules")
    new_thread_slug = new_thread["slug"]
    res = anyllm_client.chat_with_thread(workspace_slug, new_thread_slug, prompt)
    results.append({
        "response": res["textResponse"],
        "sources": res["sources"]
    })
        
        
        
    # anyllm_client.delete_thread(workspace_slug, new_thread_slug)
            
path_results = "results_rules.json"
with open(path_results, "w") as f:
    json.dump(results, f)

Creating new thread with name: rules in workspace: temperature_play
